In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Category10, Category20
from bokeh.io import export_svg
from tqdm import tqdm
output_notebook()

Loading BokehJS ...

In [2]:
from metamod.tasks import TaskSwitch, AffineCorrelatedGaussian
from metamod.trainers import two_layer_training
from metamod.networks import LinearNet
from metamod.control import TaskSwitchLinearNetEq, TaskSwitchLinearNetControl
from metamod.utils import plot_lines

Loading BokehJS ...

In [3]:
run_name = "task_switch_test"
results_path = "../results"

n_steps = 21000
save_weights_every = 20
iter_control = 50

results_dict = {}

# Init dataset
batch_size = 2048
dataset1_params = {"mu_vec": (3.0, 1.0), "sigma_vec": (1.0, 1.0), "dependence_parameter": 0.8, "batch_size": batch_size}
dataset2_params = {"mu_vec": (-2.0, 2.0), "sigma_vec": (1.0, 1.0), "dependence_parameter": 0.2, "batch_size": batch_size}
dataset_params = {"dataset1_params": dataset1_params, 
                  "dataset2_params": dataset2_params, 
                  "change_tasks_every": 1500}

model_params = {"learning_rate": 5e-3,
                "hidden_dim": 6,
                "intrinsic_noise": 0.00,
                "reg_coef": 0.01,
                "W1_0": None,
                "W2_0": None}

control_params = {"control_lower_bound": -0.5,
                  "control_upper_bound": 0.5,
                  "gamma": 0.99,
                  "cost_coef": 0.3,
                  "reward_convertion": 1.0,
                  "init_g": None,
                  "control_lr": 1.0}

In [4]:
dataset = TaskSwitch(dataset_classes=(AffineCorrelatedGaussian, AffineCorrelatedGaussian),
                     dataset_list_params=(dataset1_params, dataset2_params),
                     change_tasks_every=dataset_params["change_tasks_every"])

model_params["input_dim"] = dataset.input_dim
model_params["output_dim"] = dataset.output_dim

In [5]:
model = LinearNet(**model_params)

In [6]:
iters, loss, weights_iter, weights = two_layer_training(model=model, dataset=dataset, n_steps=n_steps,
                                                        save_weights_every=save_weights_every)

results_dict["iters"] = iters
results_dict["Loss_t_sim"] = loss
results_dict["weights_sim"] = weights
results_dict["weights_iters_sim"] = weights_iter

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21000/21000 [00:21<00:00, 993.06it/s]


In [7]:
# Solving equation
init_W1 = weights[0][0, ...]
init_W2 = weights[1][0, ...]

init_weights = [init_W1, init_W2]
input_corr, output_corr, input_output_corr, expected_y, expected_x = dataset.get_correlation_matrix()

time_span = np.arange(0, len(iters)) * model_params["learning_rate"]
results_dict["time_span"] = time_span

equation_params = {"in_cov": input_corr,
                   "out_cov": output_corr,
                   "in_out_cov": input_output_corr,
                   "init_weights": init_weights,
                   "n_steps": n_steps,
                   "reg_coef": model_params["reg_coef"],
                   "intrinsic_noise": model_params["intrinsic_noise"],
                   "learning_rate": model_params["learning_rate"],
                   "change_task_every": dataset_params["change_tasks_every"],
                   "time_constant": 1.0}

solver = TaskSwitchLinearNetEq(**equation_params)

control_params = {**control_params, **equation_params}
control = TaskSwitchLinearNetControl(**control_params)

W1_t, W2_t = solver.get_weights(time_span, get_numpy=True)
Loss_t = solver.get_loss_function(W1_t, W2_t, get_numpy=True)

results_dict["W1_t_eq"] = W1_t
results_dict["W2_t_eq"] = W2_t
results_dict["Loss_t_eq"] = Loss_t

W1_t_control, W2_t_control = control.get_weights(time_span, get_numpy=True)
Loss_t_control = control.get_loss_function(W1_t_control, W2_t_control, get_numpy=True)

In [8]:
control.control_lr

1.0

In [9]:
losses = (loss, Loss_t, Loss_t_control)
colors = (Category10[10][0], Category10[10][0], Category10[10][1])
legends = ("Simulation", "Equation", "Init Control")
alphas = (0.3, 1, 1)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

In [10]:
results_dict["W1_t_control_init"] = W1_t_control
results_dict["W2_t_control_init"] = W2_t_control
results_dict["Loss_t_control_init"] = Loss_t_control
results_dict["control_signal_init"] = (control.g1_tilda, control.g2_tilda)

control_params["iters_control"] = iter_control
cumulated_reward = []

for i in range(iter_control):
    R = control.train_step(get_numpy=True)
    print(R)
    cumulated_reward.append(R)
cumulated_reward = np.array(cumulated_reward).astype(float)
results_dict["cumulated_reward_opt"] = cumulated_reward

W1_t_opt, W2_t_opt = control.get_weights(time_span, get_numpy=True)
Loss_t_opt = control.get_loss_function(W1_t_opt, W2_t_opt, get_numpy=True)

results_dict["W1_t_control_opt"] = W1_t_opt
results_dict["W2_t_control_opt"] = W2_t_opt
results_dict["Loss_t_control_opt"] = Loss_t_opt

-22.481188
-20.425488
-20.066292
-19.590927
-19.452333
-19.364283
-19.294973
-19.238716
-19.190926
-19.148846
-19.110786
-19.075216
-19.04028
-19.003252
-18.968819
-18.943089
-18.91901
-18.894094
-18.862848
-18.82577
-18.786232
-18.746298
-18.708576
-18.673708
-18.641687
-18.612314
-18.585598
-18.563856
-18.54902
-18.58118
-18.59641
-18.67092
-18.550375
-18.45615
-18.442282
-18.41795
-18.406183
-18.384586
-18.375685
-18.354412
-18.348833
-18.326061
-18.322844
-18.298656
-18.296135
-18.27216
-18.269308
-18.246807
-18.24336
-18.222721


In [11]:
opt = plot_lines(np.arange(iter_control), (cumulated_reward,), x_axis_label="gradient steps on control", y_axis_label="Cumulated reward")
show(opt)

In [12]:
losses = (loss, Loss_t, Loss_t_opt)
colors = (Category10[10][0], Category10[10][0], Category10[10][1])
legends = ("Real Non-linear", "Approximation", "Approximated Optimized Control")
alphas = (0.3, 1, 1)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

In [13]:
g1_tilda = control.g1_tilda
g2_tilda = control.g2_tilda
control_signal = (g1_tilda, g2_tilda)
W1_0, W2_0 = control_params["init_weights"]
results_dict["control_signal"] = control_signal

In [14]:
reset_model_params = model_params.copy()
reset_model_params["W1_0"] = W1_0
reset_model_params["W2_0"] = W2_0

reset_model = LinearNet(**reset_model_params)

In [15]:
iters, loss_OPT, weights_iter_OPT, weights_OPT = two_layer_training(model=reset_model,
                                                                    dataset=dataset,
                                                                    n_steps=n_steps,
                                                                    save_weights_every=save_weights_every,
                                                                    control_signal=control_signal)
results_dict["Loss_t_sim_OPT"] = loss_OPT
results_dict["weights_sim_OPT"] = weights_OPT
results_dict["weights_iters_sim_OPT"] = weights_iter_OPT
results_dict["iters_OPT"] = iters

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21000/21000 [00:26<00:00, 805.60it/s]


In [16]:
losses = (loss, Loss_t, Loss_t_opt, loss_OPT)
colors = (Category10[10][0], Category10[10][0], Category10[10][1], Category10[10][1])
legends = ("Real Non-linear", "Approximation", "Approximated Optimized Control", "Optimized Non-linear")
alphas = (0.3, 1, 1, 0.3)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)